In [3]:
import pandas as pd
import nba_api
import bar_chart_race as bcr
import warnings
import json
warnings.simplefilter('ignore')

In [4]:
from nba_api.stats.endpoints import playergamelogs

response = playergamelogs.PlayerGameLogs(season_nullable='2021-22')
content = json.loads(response.get_json())
results = content['resultSets'][0]
headers = results['headers']
rows = results['rowSet']
df = pd.DataFrame(rows,columns= headers)

In [7]:
data = df.copy()

In [8]:
data['Date'] = pd.to_datetime(data['GAME_DATE']) - pd.to_timedelta(7, unit='d')
data = data.groupby(['PLAYER_NAME', pd.Grouper(key='Date', freq='W-MON')])['PTS'].sum().reset_index().sort_values('Date')

In [12]:
data.sort_values(by='Date',ascending=False)

,PLAYER_NAME,Date,PTS
5918,Kris Dunn,2022-04-04,9
6989,Moritz Wagner,2022-04-04,46
7596,Patrick Beverley,2022-04-04,13
1422,Chuma Okeke,2022-04-04,48
8158,Rui Hachimura,2022-04-04,72
...,...,...,...
4516,Jayson Tatum,2021-10-18,110
4491,Jaylen Nowell,2021-10-18,2
4463,Jaylen Brown,2021-10-18,85
4452,Jay Scrubb,2021-10-18,6


In [13]:
df_weekly = data.pivot_table(values = 'PTS',index = ['Date'], columns = 'PLAYER_NAME')

In [14]:
df_weekly.fillna(0, inplace=True)

In [15]:
df_weekly.iloc[:, 0:-1] = df_weekly.iloc[:, 0:-1].cumsum()

In [17]:
top_scorer = set()
for index, row in df_weekly.iterrows():
    top_scorer |= set(row[row > 0].sort_values(ascending=False).head(10).index)
df_weekly = df_weekly[top_scorer]
df_weekly.index = 'Week ' + df_weekly.index.astype(str)
df_weekly.to_csv('NBA_Points_Bar_Plot.csv')

In [18]:
df_weekly

PLAYER_NAME,Devin Booker,Zach LaVine,Ja Morant,DeMar DeRozan,Karl-Anthony Towns,Trae Young,Bradley Beal,LeBron James,Stephen Curry,Kevin Durant,...,Domantas Sabonis,Jayson Tatum,Paul George,Miles Bridges,Luka Doncic,Nikola Jokic,Malcolm Brogdon,Donovan Mitchell,Brandon Ingram,Giannis Antetokounmpo
Date,,,,,,,,,,,,,,,,,,,,,
Week 2021-10-18,55.0,102.0,105.0,90.0,87.0,75.0,42.0,78.0,93.0,124.0,...,99.0,110.0,84.0,100.0,45.0,83.0,99.0,43.0,108.0,98.0
Week 2021-10-25,113.0,179.0,198.0,179.0,149.0,160.0,146.0,119.0,172.0,194.0,...,170.0,180.0,170.0,185.0,135.0,167.0,117.0,138.0,150.0,191.0
Week 2021-11-01,214.0,262.0,260.0,269.0,212.0,261.0,218.0,149.0,276.0,324.0,...,212.0,236.0,243.0,267.0,249.0,254.0,134.0,232.0,150.0,273.0
Week 2021-11-08,298.0,363.0,362.0,377.0,301.0,383.0,233.0,149.0,365.0,415.0,...,277.0,329.0,344.0,323.0,324.0,339.0,224.0,307.0,200.0,345.0
Week 2021-11-15,381.0,467.0,433.0,474.0,397.0,450.0,326.0,182.0,454.0,484.0,...,341.0,463.0,449.0,395.0,324.0,369.0,292.0,384.0,257.0,457.0
Week 2021-11-22,478.0,557.0,458.0,570.0,493.0,545.0,413.0,284.0,544.0,544.0,...,398.0,510.0,544.0,455.0,408.0,393.0,372.0,464.0,342.0,540.0
Week 2021-11-29,488.0,647.0,458.0,633.0,558.0,657.0,494.0,307.0,637.0,629.0,...,464.0,604.0,599.0,525.0,457.0,460.0,430.0,533.0,435.0,607.0
Week 2021-12-06,488.0,703.0,458.0,633.0,624.0,729.0,559.0,420.0,703.0,735.0,...,539.0,733.0,599.0,575.0,538.0,584.0,481.0,619.0,504.0,703.0
Week 2021-12-13,504.0,703.0,474.0,697.0,708.0,791.0,652.0,493.0,785.0,803.0,...,567.0,802.0,624.0,655.0,538.0,631.0,493.0,726.0,560.0,703.0


In [19]:
bcr.bar_chart_race(df=df_weekly, 
                   n_bars=15, 
                   sort='desc',
                   title='NBA POINTS LEADERS BY WEEK',
                   period_length=1000,
                   interpolate_period=False,
                   steps_per_period=10,
                   fixed_max=True)